In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import statsmodels.api as sm

In [2]:
### zbp total with features data
file_path = '../../../src/data/temp/lagged_zbp_totals_with_features.csv'
data = pd.read_csv(file_path)

## Correlation Matrix

In [3]:
numeric_columns = data.select_dtypes(include=['number']).columns
selected_df = data[numeric_columns]

correlation_matrix = selected_df.corr()

In [4]:
correlation_matrix.head()

,zip,year,est,emp,qp1,ap,naics_11_pct,naics_21_pct,naics_22_pct,naics_23_pct,...,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct,median_hh_income,total_population,total_retirement,total_midcareer (25-34),total_midcareer (35-44),est_lag_1
zip,1.000000e+00,2.324927e-13,0.252894,0.255591,0.239795,0.239811,-0.042056,-0.098355,-0.086965,-0.269077,...,0.162904,0.036867,0.209696,0.037120,0.002810,0.181370,0.099167,0.232368,0.233891,0.250942
year,2.324927e-13,1.000000e+00,0.056778,0.039271,0.073006,0.068126,-0.107906,-0.238683,-0.165284,0.101599,...,-0.265407,-0.247510,-0.291691,-0.107464,0.254769,0.037928,0.300165,-0.329487,0.159178,0.057180
est,2.528937e-01,5.677763e-02,1.000000,0.863791,0.694837,0.700478,-0.211921,-0.039383,-0.179668,-0.398984,...,0.435856,0.218022,0.406468,-0.065021,0.086039,0.376423,0.418920,0.321703,0.418891,0.999631
emp,2.555906e-01,3.927093e-02,0.863791,1.000000,0.919969,0.925050,-0.163817,-0.032267,-0.084435,-0.330160,...,0.595323,0.376813,0.578313,-0.005200,0.071532,0.139771,0.154111,0.165351,0.202244,0.866002
qp1,2.397954e-01,7.300614e-02,0.694837,0.919969,1.000000,0.998518,-0.114618,-0.030377,-0.041629,-0.262233,...,0.515435,0.374632,0.518620,0.014935,0.179208,-0.007138,0.018123,0.006089,0.050494,0.697360


In [5]:
correlation_with_est = correlation_matrix['est'].sort_values(ascending=False)

top_5_features = correlation_with_est.head(6)[1:]  # Excluding 'est' 

print(top_5_features)

est_lag_1     0.999631
emp           0.863791
ap            0.700478
qp1           0.694837
n20_49_pct    0.556862
Name: est, dtype: float64


# DROP NON-NUMERICAL

In [6]:
non_numerical_cols = data.select_dtypes(exclude=['int64', 'float64']).columns
data = data.drop(columns=non_numerical_cols)
data.head(1)

,zip,year,est,emp,qp1,ap,naics_11_pct,naics_21_pct,naics_22_pct,naics_23_pct,...,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct,median_hh_income,total_population,total_retirement,total_midcareer (25-34),total_midcareer (35-44),est_lag_1
0,91901,2013,402,4141.0,36304.0,174786.0,0.0,0.0,0.0,0.225064,...,0.005115,0.002558,0.0,0.002558,76496.0,17034.0,2691.0,1441.0,2011.0,391.0


# TRAIN-TEST SPLIT

In [7]:
end_year = 2020
data_train = data[data['year'] <= end_year]
data_test = data[data['year'] > end_year]

# STANDARDIZATION

In [8]:
train_mean = data_train.mean()
train_mean.loc['zip'] = 0

train_std = data_train.std()
train_std.loc['zip'] = 1

In [9]:
data_train = (data_train - train_mean) / train_std
data_train.head(1)

,zip,year,est,emp,qp1,ap,naics_11_pct,naics_21_pct,naics_22_pct,naics_23_pct,...,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct,median_hh_income,total_population,total_retirement,total_midcareer (25-34),total_midcareer (35-44),est_lag_1
0,91901.0,-1.526464,-0.69421,-0.555844,-0.420879,-0.4128,-0.318008,-0.326679,-0.255655,1.298759,...,-0.569958,0.03877,-0.474458,0.309908,0.064899,-0.864003,-0.721704,-0.858575,-0.908728,-0.696389


In [10]:
data_test = (data_test - train_mean) / train_std
data_test.head(1)

,zip,year,est,emp,qp1,ap,naics_11_pct,naics_21_pct,naics_22_pct,naics_23_pct,...,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct,median_hh_income,total_population,total_retirement,total_midcareer (25-34),total_midcareer (35-44),est_lag_1
8,91901.0,1.962597,-0.656603,-0.48519,-0.36702,-0.37546,-0.318008,-0.326679,-0.255655,1.56997,...,-1.059046,-0.504829,-0.474458,-0.154413,0.713168,-0.823683,-0.183901,-0.987038,-0.83604,-0.631105


# MODEL

In [11]:
preproc = ColumnTransformer([('onehots', OneHotEncoder(handle_unknown='ignore'), ['zip'])]
                             ,remainder = 'passthrough')
pl = Pipeline(steps=[('preproc', preproc), ('lr', LinearRegression(n_jobs=-1))])

# TESTING

In [12]:
def unstandardize_series(ser, mean, std):
    return (ser*std)+mean

In [13]:
def fit_eval(model, data_train, data_test, included_feats):
    X_train = data_train[included_feats]
    y_train = data_train['est']
    X_test = data_test[included_feats]
    y_test = data_test['est']
    
    pl.fit(X_train, y_train)
    
    y_preds = pl.predict(X_train)
    inverted_y_train = unstandardize_series(y_train, train_mean['est'], train_std['est'])
    inverted_y_preds = unstandardize_series(y_preds, train_mean['est'], train_std['est'])
    train_rmse = mean_squared_error(inverted_y_train, inverted_y_preds, squared=False)
    
    y_preds = pl.predict(X_test)
    inverted_y_test = unstandardize_series(y_test, train_mean['est'], train_std['est'])
    inverted_y_preds = unstandardize_series(y_preds, train_mean['est'], train_std['est'])
    test_rmse = mean_squared_error(inverted_y_test, inverted_y_preds, squared=False)
    
    return pl, train_rmse, test_rmse

# Using All Features

In [14]:
pl, train_rmse, test_rmse = fit_eval(pl, data_train, data_test, data.columns.drop(['est']))
print('train_rmse: ', train_rmse)
print('test_rmse: ', test_rmse)

train_rmse:  14.89858677502568
test_rmse:  21.506695364323548


## Using the Top 5 Features From Correlation Matrix

In [15]:
top_5_features = correlation_with_est.head(6)[1:]
included_feats = top_5_features.index.append(pd.Index(['zip']))

pl, train_rmse, test_rmse = fit_eval(pl, data_train, data_test, included_feats)
print('train_rmse: ', train_rmse)
print('test_rmse: ', test_rmse)

train_rmse:  15.837656195446769
test_rmse:  22.166332122670465


## Using the Top 10 Features

In [16]:
top_10_features = correlation_with_est.head(11)[1:]
included_feats = top_10_features.index.append(pd.Index(['zip']))

pl, train_rmse, test_rmse = fit_eval(pl, data_train, data_test, included_feats)
print('train_rmse: ', train_rmse)
print('test_rmse: ', test_rmse)

train_rmse:  15.59438038294449
test_rmse:  21.303388104681762


# Fixed Effect Model

In [19]:
# Standardize Data
std_data = (data-train_mean)/train_std
std_data = data

# Create dummy variables for each ZIP code
dummies = pd.get_dummies(std_data['zip'], drop_first=True).astype(int)

# Concatenate the dummy variables with the original data
data_panel = pd.concat([std_data.drop(columns=['zip']), dummies], axis=1)

X = data_panel.drop(columns=['est']) 
y = data_panel['est']  # Dependent variable predicting establishment growth

# Constant term to the independent variables
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    est   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.254e+04
Date:                Wed, 14 Feb 2024   Prob (F-statistic):               0.00
Time:                        21:59:21   Log-Likelihood:                -3368.1
No. Observations:                 810   AIC:                             6992.
Df Residuals:                     682   BIC:                             7593.
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -5

## Statistically Significant Values

- em with a p-value of 0.0000
- ap with a p-value of 0.006
- n1_4_pct with a p-value < 0.000
- n5_9_pct with a p-value < 0.000
- n10_19_pct with a p-value < 0.000
- n20_49_pct with a p-value < 0.000
- n50_99_pct with a p-value < 0.000

## Interesting Zip Code

### - 91910 p-value (0.744)
### - 91911 p-value (0.385)

This output indicates that the model has a high R-squared value **(0.998)**, suggesting that the independent variables explain a large portion of the variance in the dependent variable.

# Random Effect Model

In [18]:
X = data_panel.drop(columns=['est']) 
y = data_panel['est']  

X = sm.add_constant(X)

model = sm.regression.mixed_linear_model.MixedLM(y, X, groups=std_data['zip'])

# Specify the optimizer (e.g., 'nm' for Nelder-Mead)
optimizer = 'nm'

iterations = 1000

mixed_model_fit = model.fit(method=optimizer, maxiter=iterations)

print(mixed_model_fit.summary())

                    Mixed Linear Model Regression Results
Model:                     MixedLM        Dependent Variable:        est      
No. Observations:          810            Method:                    REML     
No. Groups:                90             Scale:                     0.0005   
Min. group size:           9              Log-Likelihood:            1451.8098
Max. group size:           9              Converged:                 Yes      
Mean group size:           9.0                                                
------------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|    [0.025     0.975]  
------------------------------------------------------------------------------
const                    -0.087      0.031 -2.826 0.005      -0.147     -0.027
year                      0.009      0.002  4.448 0.000       0.005      0.013
emp                      -0.034      0.013 -2.600 0.009      -0.060     -

C:\Users\Innominandum\anaconda3\envs\dsc180-capstone\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\Users\Innominandum\anaconda3\envs\dsc180-capstone\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


## Statistically Significant Values

- ap with p-value 0.006
- n1_4_pct with p-value 0.000
- n5_9_pct with p-value 0.000
- n10_19_pct with p-value 0.000
- n20_49_pct with p-value 0.000
- All zipcodes

## Summary 

**Significance of the individual-specific effects**: The p-values for the individual-specific effects in the fixed effects model are statistically significant, indicating that there is likely unobserved heterogeneity at the individual level that affects the outcome variable. Suggesting that there are **individual-specific** characteristics or factors that are important to consider and control for in your analysis.

**Adjusted R-squared**: The adjusted R-squared value for the fixed effects model is higher compared to the random effects model. This suggests that the fixed effects model explains a greater proportion of the variation in the outcome variable.

Overall **fixed effect model** seems better for the data